<h3>Creating and Training the Convolutoinal Neural Network Models</h3> 

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pandas as pd
from torch.utils.data import WeightedRandomSampler
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim 
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from ignite.engine import Engine, Events
from ignite.handlers import EarlyStopping
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, ConfusionMatrix, Precision, Recall
from ignite.contrib.metrics.regression import R2Score, CanberraMetric, MeanError
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine
import numpy as np
import torchvision
from sklearn.metrics import confusion_matrix
from torchvision import datasets, models, transforms
from torchmetrics.functional import r2_score
import matplotlib.pyplot as plt
import os

cudnn.benchmark = True
plt.ion()   # 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Create a custom dataset class to load in the image data. To create a dataset, you need a CSV annotations file as well as a directory of images.

In [2]:
class ClusterDataSet(Dataset):
    
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform    
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[index, 0])
        image = np.array(PIL.Image.open(img_path).convert('RGB'))        
        label = self.img_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


Function to calculate the mean and standard deviation from a dataloader

In [3]:
def mean_std(dataloader):
    channels_added = 0
    channels_squared_added = 0
    n_batches = 0
    for x, _ in dataloader:
        channels_added += torch.mean(torch.FloatTensor(x), dim=[0,2,3])
        channels_squared_added += torch.mean(x**2, dim=[0,2,3])
        n_batches += 1
    mean = channels_added / n_batches
    standard_deviation = (channels_squared_added / n_batches - mean ** 2) ** 0.5
    return mean, standard_deviation

Create a dataset solely to determine the mean and standard deviation of the training dataset. These values will be used to normalize the data.

In [4]:
train_not_normal_dataset = ClusterDataSet(annotations_file='train_annotations.csv', img_dir='all_images_final', transform=transforms.ToTensor(), target_transform=int)
not_normal_dataloader = DataLoader(dataset=train_not_normal_dataset, batch_size=16, shuffle=True, num_workers=0)
mean, std = mean_std(not_normal_dataloader)

In [10]:
print(mean)
print(std)

tensor([0.3560, 0.3692, 0.3417])
tensor([0.2059, 0.1733, 0.1620])


Now create the actual training and testing dataloaders. These will be used to train and validate the different machine learning models. 

In [6]:
tfs_training = transforms.Compose(
        [
            transforms.ToTensor(), 
            transforms.Normalize(mean=mean, std=std), 
            transforms.RandomHorizontalFlip(), 
            transforms.RandomResizedCrop(224)
        ]
    )

tfs_val = transforms.Compose(
        [
            transforms.ToTensor(), 
            transforms.Normalize(mean=mean, std=std) 
        ]
    )

import torch.utils.data as data_utils

normalized_train = ClusterDataSet(annotations_file='train_annotations.csv', img_dir='all_images_final', transform=tfs_training, target_transform=int)
normalized_test = ClusterDataSet(annotations_file='all_val_annotations.csv', img_dir='all_val_images', transform=tfs_training, target_transform=int)    

part_tr = torch.utils.data.random_split(normalized_train, [len(normalized_train), 0])[0]
part_te = torch.utils.data.random_split(normalized_test, [len(normalized_test)-13000, 13000])[0]

Create the training and testing dataloaders

In [7]:
train_dataloader = DataLoader(dataset=part_tr, batch_size=16, num_workers=0)
test_dataloader = DataLoader(dataset=part_te, batch_size=16, num_workers=0)

Define the VGG model. This is the first of 3 different models I defined.

In [14]:
vgg_model = models.vgg13(weights=models.VGG13_Weights.IMAGENET1K_V1)
training_precision_vgg = []
validation_precision_vgg = []
training_recall_vgg = []
validation_recall_vgg = []

classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=4096, out_features=384),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=384, out_features=3)
)

vgg_model.classifier = classifier
vgg_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg_model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

def train_step(engine, batch):
    vgg_model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = vgg_model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)


val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion),
    "precision": Precision(),
    "recall": Recall()
}
confusion = ConfusionMatrix(num_classes=3)

train_evaluator = create_supervised_evaluator(vgg_model, metrics=val_metrics, device=device)
val_evaluator = create_supervised_evaluator(vgg_model, metrics=val_metrics, device=device)
confusion.attach(train_evaluator, 'confusion matrix')
confusion.attach(val_evaluator, 'confusion matrix')

log_interval = 100

@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f} ")


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    print()
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Training Precision - Epoch[{trainer.state.epoch}] Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Training Recall - Epoch[{trainer.state.epoch}] Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f} ')
    print()
    training_precision_vgg.append(precision)
    training_recall_vgg.append(recall)

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(test_dataloader)
    metrics = val_evaluator.state.metrics
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    print(f'Validation Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Validation Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    print()
    print('Confusion Matrix')
    print(f'{metrics["confusion matrix"]}')
    print()
    validation_precision_vgg.append(precision)
    validation_recall_vgg.append(recall)


def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint = ModelCheckpoint(
    "VGG_FINAL",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer),
    require_empty=False
)
  
early_stopper = EarlyStopping(patience=5, score_function=score_function, trainer=trainer)

val_evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": vgg_model})
val_evaluator.add_event_handler(Events.COMPLETED, early_stopper)

tb_logger = TensorboardLogger(log_dir="tb-logger-VGG-FINAL")

tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

for tag, evaluator in [("training", train_evaluator), ("validation", val_evaluator)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer),
    )

Run the model

In [11]:
trainer.run(train_dataloader, max_epochs=30)

Epoch[1], Iter[100] Loss: 0.49 
Epoch[1], Iter[200] Loss: 0.42 
Epoch[1], Iter[300] Loss: 0.48 
Epoch[1], Iter[400] Loss: 0.43 
Epoch[1], Iter[500] Loss: 0.81 
Epoch[1], Iter[600] Loss: 0.47 
Epoch[1], Iter[700] Loss: 0.37 
Epoch[1], Iter[800] Loss: 0.72 
Epoch[1], Iter[900] Loss: 0.38 
Epoch[1], Iter[1000] Loss: 0.45 
Epoch[1], Iter[1100] Loss: 0.70 
Epoch[1], Iter[1200] Loss: 0.45 
Epoch[1], Iter[1300] Loss: 0.28 
Epoch[1], Iter[1400] Loss: 0.40 
Epoch[1], Iter[1500] Loss: 0.14 
Epoch[1], Iter[1600] Loss: 0.46 
Epoch[1], Iter[1700] Loss: 0.22 
Epoch[1], Iter[1800] Loss: 0.37 


c:\Python310\lib\site-packages\ignite\contrib\handlers\base_logger.py:172: UserWarning: Logger output_handler can not log metrics value type <class 'torch.Tensor'>
  warnings.warn(f"Logger output_handler can not log metrics value type {type(value)}")



Training Results - Epoch[1] Avg accuracy: 0.82 Avg loss: 0.41
Training Precision - Epoch[1] Low Nightlight: 0.92 Medium Nightlight: 0.53 High Nightlight: 0.61 
Training Recall - Epoch[1] Low Nightlight: 0.93 Medium Nightlight: 0.34 High Nightlight: 0.86 

Validation Results - Epoch[1] Avg accuracy: 0.67 Avg loss: 0.48
Validation Precision - Low Nightlight: 1.00 Medium Nightlight: 0.00 High Nightlight: 0.00 
Validation Recall - Low Nightlight: 1.00 Medium Nightlight: 0.00 High Nightlight: 0.00

Confusion Matrix
tensor([[2, 0, 0],
        [0, 0, 1],
        [0, 0, 0]])

Epoch[2], Iter[1900] Loss: 0.38 
Epoch[2], Iter[2000] Loss: 0.76 
Epoch[2], Iter[2100] Loss: 0.20 
Epoch[2], Iter[2200] Loss: 0.31 
Epoch[2], Iter[2300] Loss: 0.40 
Epoch[2], Iter[2400] Loss: 0.49 
Epoch[2], Iter[2500] Loss: 0.30 
Epoch[2], Iter[2600] Loss: 0.42 
Epoch[2], Iter[2700] Loss: 0.51 
Epoch[2], Iter[2800] Loss: 0.45 
Epoch[2], Iter[2900] Loss: 0.29 
Epoch[2], Iter[3000] Loss: 0.56 
Epoch[2], Iter[3100] Loss: 0

2023-04-18 22:56:02,496 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training



Training Results - Epoch[6] Avg accuracy: 0.87 Avg loss: 0.33
Training Precision - Epoch[6] Low Nightlight: 0.93 Medium Nightlight: 0.67 High Nightlight: 0.74 
Training Recall - Epoch[6] Low Nightlight: 0.96 Medium Nightlight: 0.50 High Nightlight: 0.83 

Validation Results - Epoch[6] Avg accuracy: 0.67 Avg loss: 0.28
Validation Precision - Low Nightlight: 1.00 Medium Nightlight: 0.00 High Nightlight: 0.00 
Validation Recall - Low Nightlight: 1.00 Medium Nightlight: 0.00 High Nightlight: 0.00

Confusion Matrix
tensor([[2, 0, 0],
        [0, 0, 1],
        [0, 0, 0]])



State:
	iteration: 10830
	epoch: 6
	epoch_length: 1805
	max_epochs: 30
	output: 0.2805481553077698
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

Define transformer model

In [15]:
transformer_model = models.vit_b_16(weights='DEFAULT')
# classifier2 = nn.Sequential(nn.Linear(in_features=768, out_features=3))
training_precision_vit = []
validation_precision_vit = []
training_recall_vit = []
validation_recall_vit = []

classifier2 = nn.Sequential(
    nn.Linear(in_features=768, out_features=384),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=384, out_features=384),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=384, out_features=3)
)

transformer_model.heads = classifier2
transformer_model = transformer_model.to(device)
criterion_vit = nn.CrossEntropyLoss()
optimizer_vit = optim.SGD(transformer_model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler_vit = lr_scheduler.StepLR(optimizer_vit, step_size=2, gamma=0.1)

trainer_vit = create_supervised_trainer(transformer_model, optimizer_vit, criterion_vit, device)


val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion_vit),
    "precision": Precision(),
    "recall": Recall()
}


train_evaluator_vit = create_supervised_evaluator(transformer_model, metrics=val_metrics, device=device)
val_evaluator_vit = create_supervised_evaluator(transformer_model, metrics=val_metrics, device=device)

confusion = ConfusionMatrix(num_classes=3)
confusion.attach(train_evaluator_vit, 'confusion matrix')
confusion.attach(val_evaluator_vit, 'confusion matrix')

log_interval = 100

def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


trainer_vit.add_event_handler(Events.ITERATION_COMPLETED(every=log_interval), log_training_loss)


def log_training_results(trainer):
    train_evaluator_vit.run(train_dataloader)
    metrics = train_evaluator_vit.state.metrics
    print()
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Training Precision - Epoch[{trainer.state.epoch}] Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Training Recall - Epoch[{trainer.state.epoch}] Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f} ')
    training_precision_vit.append(precision)
    training_precision_vit.append(recall)
    print()

trainer_vit.add_event_handler(Events.EPOCH_COMPLETED, log_training_results)


def log_validation_results(trainer):
    val_evaluator_vit.run(test_dataloader)
    metrics = val_evaluator_vit.state.metrics
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    print(f'Training Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Training Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')

    print('Confusion Matrix')
    print(f'{metrics["confusion matrix"]}')
    validation_precision_vit.append(precision)
    validation_precision_vit.append(recall)
    print()

trainer_vit.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)


def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint_vit = ModelCheckpoint(
    "Vit_REAL",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer_vit),
    require_empty=False
)
  
early_stopper = EarlyStopping(patience=4, score_function=score_function, trainer=trainer_vit)

val_evaluator_vit.add_event_handler(Events.COMPLETED, model_checkpoint_vit, {"model": transformer_model})
val_evaluator_vit.add_event_handler(Events.COMPLETED, early_stopper)

tb_logger = TensorboardLogger(log_dir="tb-logger-VIT_REAL")

tb_logger.attach_output_handler(
    trainer_vit,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

for tag, evaluator in [("training", train_evaluator_vit), ("validation", val_evaluator_vit)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer_vit),
    )

Run transfomer model

In [16]:
trainer_vit.run(train_dataloader, max_epochs = 30)

Epoch[1], Iter[100] Loss: 1.18
Epoch[1], Iter[200] Loss: 0.61
Epoch[1], Iter[300] Loss: 0.28
Epoch[1], Iter[400] Loss: 0.47
Epoch[1], Iter[500] Loss: 0.27
Epoch[1], Iter[600] Loss: 0.25
Epoch[1], Iter[700] Loss: 0.64
Epoch[1], Iter[800] Loss: 0.54
Epoch[1], Iter[900] Loss: 0.77
Epoch[1], Iter[1000] Loss: 0.59
Epoch[1], Iter[1100] Loss: 0.16
Epoch[1], Iter[1200] Loss: 0.15
Epoch[1], Iter[1300] Loss: 0.58
Epoch[1], Iter[1400] Loss: 0.37
Epoch[1], Iter[1500] Loss: 0.20
Epoch[1], Iter[1600] Loss: 0.32
Epoch[1], Iter[1700] Loss: 0.44
Epoch[1], Iter[1800] Loss: 0.22


c:\Python310\lib\site-packages\ignite\contrib\handlers\base_logger.py:172: UserWarning: Logger output_handler can not log metrics value type <class 'torch.Tensor'>
  warnings.warn(f"Logger output_handler can not log metrics value type {type(value)}")



Training Results - Epoch[1] Avg accuracy: 0.83 Avg loss: 0.40
Training Precision - Epoch[1] Low Nightlight: 0.92 Medium Nightlight: 0.57 High Nightlight: 0.65 
Training Recall - Epoch[1] Low Nightlight: 0.94 Medium Nightlight: 0.40 High Nightlight: 0.83 

Validation Results - Epoch[1] Avg accuracy: 0.87 Avg loss: 0.35
Training Precision - Low Nightlight: 0.93 Medium Nightlight: 0.59 High Nightlight: 0.74 
Training Recall - Low Nightlight: 0.95 Medium Nightlight: 0.42 High Nightlight: 0.84
Confusion Matrix
tensor([[697,  27,   6],
        [ 44,  57,  34],
        [  9,  13, 116]])

Epoch[2], Iter[1900] Loss: 0.21
Epoch[2], Iter[2000] Loss: 0.60
Epoch[2], Iter[2100] Loss: 0.38
Epoch[2], Iter[2200] Loss: 0.56
Epoch[2], Iter[2300] Loss: 0.47
Epoch[2], Iter[2400] Loss: 0.73
Epoch[2], Iter[2500] Loss: 0.27
Epoch[2], Iter[2600] Loss: 0.42
Epoch[2], Iter[2700] Loss: 0.28
Epoch[2], Iter[2800] Loss: 0.50
Epoch[2], Iter[2900] Loss: 0.66
Epoch[2], Iter[3000] Loss: 0.48
Epoch[2], Iter[3100] Loss: 

2023-04-19 04:26:48,833 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Validation Results - Epoch[11] Avg accuracy: 0.91 Avg loss: 0.24
Training Precision - Low Nightlight: 0.95 Medium Nightlight: 0.69 High Nightlight: 0.85 
Training Recall - Low Nightlight: 0.97 Medium Nightlight: 0.59 High Nightlight: 0.85
Confusion Matrix
tensor([[711,  19,   0],
        [ 35,  80,  20],
        [  4,  17, 117]])



State:
	iteration: 19855
	epoch: 11
	epoch_length: 1805
	max_epochs: 30
	output: 0.04028015211224556
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

Define resnet model

In [17]:
resnet_model = models.resnet18(weights='DEFAULT')
training_precision_resnet = []
validation_precision_resnet = []
training_recall_resnet = []
validation_recall_resnet = []

classifier3 = nn.Sequential(
    nn.Linear(in_features=512, out_features=512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=384),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=384, out_features=3)
)

resnet_model.fc = classifier3
resnet_model = resnet_model.to(device)
criterion_resnet = nn.CrossEntropyLoss()
optimizer_resnet = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)
exp_res_scheduler_res = lr_scheduler.StepLR(optimizer_resnet, step_size=2, gamma=0.1)

trainer_resnet = create_supervised_trainer(resnet_model, optimizer_resnet, criterion_resnet, device)


val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion_resnet),
    "precision": Precision(),
    "recall": Recall()
}


train_evaluator_res = create_supervised_evaluator(resnet_model, metrics=val_metrics, device=device)
val_evaluator_res = create_supervised_evaluator(resnet_model, metrics=val_metrics, device=device)

confusion = ConfusionMatrix(num_classes=3)
confusion.attach(train_evaluator_res, 'confusion matrix')
confusion.attach(val_evaluator_res, 'confusion matrix')

log_interval = 100

def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


trainer_resnet.add_event_handler(Events.ITERATION_COMPLETED(every=log_interval), log_training_loss)


def log_training_results(trainer):
    train_evaluator_res.run(train_dataloader)
    metrics = train_evaluator_res.state.metrics
    print()
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Training Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Training Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    training_precision_resnet.append(precision)
    training_recall_resnet.append(recall)
    print()


trainer_resnet.add_event_handler(Events.EPOCH_COMPLETED, log_training_results)


def log_validation_results(trainer):
    val_evaluator_res.run(test_dataloader)
    metrics = val_evaluator_res.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Validation Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Validation Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    print()
    print('Confusion Matrix')
    print(f'{metrics["confusion matrix"]}')
    validation_precision_resnet.append(precision)
    validation_recall_resnet.append(recall)
    print()

trainer_resnet.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)


def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint_res = ModelCheckpoint(
    "res_FINAL",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer_resnet),
    require_empty=False
)
  
early_stopper = EarlyStopping(patience=6, score_function=score_function, trainer=trainer_resnet)

val_evaluator_res.add_event_handler(Events.COMPLETED, model_checkpoint_res, {"model": resnet_model})
val_evaluator_res.add_event_handler(Events.COMPLETED, early_stopper)

tb_logger = TensorboardLogger(log_dir="tb-logger-resnet_FINAL")

tb_logger.attach_output_handler(
    trainer_resnet,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

for tag, evaluator in [("training", train_evaluator_res), ("validation", val_evaluator_res)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer_resnet)
    )

In [18]:
trainer_resnet.run(train_dataloader, max_epochs=7)

Epoch[1], Iter[100] Loss: 1.45
Epoch[1], Iter[200] Loss: 0.70
Epoch[1], Iter[300] Loss: 0.36
Epoch[1], Iter[400] Loss: 0.45
Epoch[1], Iter[500] Loss: 0.32
Epoch[1], Iter[600] Loss: 0.34
Epoch[1], Iter[700] Loss: 0.81
Epoch[1], Iter[800] Loss: 0.42
Epoch[1], Iter[900] Loss: 0.65
Epoch[1], Iter[1000] Loss: 0.52
Epoch[1], Iter[1100] Loss: 0.32
Epoch[1], Iter[1200] Loss: 0.23
Epoch[1], Iter[1300] Loss: 0.42
Epoch[1], Iter[1400] Loss: 0.34
Epoch[1], Iter[1500] Loss: 0.25
Epoch[1], Iter[1600] Loss: 0.28
Epoch[1], Iter[1700] Loss: 0.48
Epoch[1], Iter[1800] Loss: 0.24

Training Results - Epoch[1] Avg accuracy: 0.84 Avg loss: 0.39
Training Precision - Low Nightlight: 0.92 Medium Nightlight: 0.57 High Nightlight: 0.68 
Training Recall - Low Nightlight: 0.94 Medium Nightlight: 0.42 High Nightlight: 0.80

Validation Results - Epoch[1] Avg accuracy: 0.87 Avg loss: 0.34
Validation Precision - Low Nightlight: 0.92 Medium Nightlight: 0.62 High Nightlight: 0.77 
Validation Recall - Low Nightlight: 0.97

State:
	iteration: 12635
	epoch: 7
	epoch_length: 1805
	max_epochs: 7
	output: 0.07449763268232346
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

Run resnet model

In [19]:
efficientnet_model = models.efficientnet_b7(weights='DEFAULT')
efficientnet_model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2560, out_features=1000, bias=True)
)

In [20]:
efficientnet_model = models.efficientnet_b3(weights='DEFAULT')
training_precision_efficientnet = []
validation_precision_efficientnet = []
training_recall_efficientnet = []
validation_recall_efficientnet = []

classifier4 = nn.Sequential(
    nn.Linear(in_features=1536, out_features=768),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=768, out_features=384),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=384, out_features=3)
)

efficientnet_model.classifier = classifier4
efficientnet_model = efficientnet_model.to(device)
criterion_efficientnet = nn.CrossEntropyLoss()
optimizer_efficientnet = optim.SGD(efficientnet_model.parameters(), lr=0.001, momentum=0.9)

trainer_efficientnet = create_supervised_trainer(efficientnet_model, optimizer_efficientnet, criterion_efficientnet, device)


val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion_efficientnet),
    "precision": Precision(),
    "recall": Recall()
}


train_evaluator_efficient = create_supervised_evaluator(efficientnet_model, metrics=val_metrics, device=device)
val_evaluator_efficient = create_supervised_evaluator(efficientnet_model, metrics=val_metrics, device=device)

confusion = ConfusionMatrix(num_classes=3)
confusion.attach(train_evaluator_efficient, 'confusion matrix')
confusion.attach(val_evaluator_efficient, 'confusion matrix')

log_interval = 100

def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


trainer_efficientnet.add_event_handler(Events.ITERATION_COMPLETED(every=log_interval), log_training_loss)


def log_training_results(trainer):
    train_evaluator_efficient.run(train_dataloader)
    metrics = train_evaluator_efficient.state.metrics
    print()
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Training Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Training Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    training_precision_efficientnet.append(precision)
    training_recall_efficientnet.append(recall)
    print()


trainer_efficientnet.add_event_handler(Events.EPOCH_COMPLETED, log_training_results)


def log_validation_results(trainer):
    val_evaluator_efficient.run(test_dataloader)
    metrics = val_evaluator_efficient.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Validation Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Validation Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    print()
    print('Confusion Matrix')
    print(f'{metrics["confusion matrix"]}')
    validation_precision_efficientnet.append(precision)
    validation_recall_efficientnet.append(recall)
    print()

trainer_efficientnet.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)


def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint_eff = ModelCheckpoint(
    "efficient_FINAL",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer_efficientnet),
    require_empty=False
)
  
early_stopper = EarlyStopping(patience=6, score_function=score_function, trainer=trainer_efficientnet)

val_evaluator_efficient.add_event_handler(Events.COMPLETED, model_checkpoint_eff, {"model": efficientnet_model})
val_evaluator_efficient.add_event_handler(Events.COMPLETED, early_stopper)

tb_logger = TensorboardLogger(log_dir="tb-logger-efficientnet_FINAL")

tb_logger.attach_output_handler(
    trainer_efficientnet,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

for tag, evaluator in [("training", train_evaluator_efficient), ("validation", val_evaluator_efficient)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer_efficientnet)
    )

In [21]:
trainer_efficientnet.run(train_dataloader, max_epochs=7)

Epoch[1], Iter[100] Loss: 1.27
Epoch[1], Iter[200] Loss: 0.68
Epoch[1], Iter[300] Loss: 0.47
Epoch[1], Iter[400] Loss: 0.66
Epoch[1], Iter[500] Loss: 0.47
Epoch[1], Iter[600] Loss: 0.41
Epoch[1], Iter[700] Loss: 0.68
Epoch[1], Iter[800] Loss: 0.72
Epoch[1], Iter[900] Loss: 0.70
Epoch[1], Iter[1000] Loss: 0.53
Epoch[1], Iter[1100] Loss: 0.43
Epoch[1], Iter[1200] Loss: 0.35
Epoch[1], Iter[1300] Loss: 0.50
Epoch[1], Iter[1400] Loss: 0.54
Epoch[1], Iter[1500] Loss: 0.33
Epoch[1], Iter[1600] Loss: 0.29
Epoch[1], Iter[1700] Loss: 0.52
Epoch[1], Iter[1800] Loss: 0.30

Training Results - Epoch[1] Avg accuracy: 0.79 Avg loss: 0.52
Training Precision - Low Nightlight: 0.90 Medium Nightlight: 0.45 High Nightlight: 0.53 
Training Recall - Low Nightlight: 0.95 Medium Nightlight: 0.12 High Nightlight: 0.85

Validation Results - Epoch[1] Avg accuracy: 0.84 Avg loss: 0.42
Validation Precision - Low Nightlight: 0.90 Medium Nightlight: 0.50 High Nightlight: 0.64 
Validation Recall - Low Nightlight: 0.97

State:
	iteration: 12635
	epoch: 7
	epoch_length: 1805
	max_epochs: 7
	output: 0.1146414652466774
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [8]:
swin_model = models.swin_b(weights='DEFAULT')
training_precision_swin = []
validation_precision_swin = []
training_recall_swin = []
validation_recall_swin = []

classifier5 = nn.Sequential(
    nn.Linear(in_features=1024, out_features=512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=3)
)

swin_model.head = classifier5
swin_model = swin_model.to(device)
criterion_swin = nn.CrossEntropyLoss()
optimizer_swin = optim.SGD(swin_model.parameters(), lr=0.001, momentum=0.9)

trainer_swin = create_supervised_trainer(swin_model, optimizer_swin, criterion_swin, device)


val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion_swin),
    "precision": Precision(),
    "recall": Recall()
}


train_evaluator_swin = create_supervised_evaluator(swin_model, metrics=val_metrics, device=device)
val_evaluator_swin = create_supervised_evaluator(swin_model, metrics=val_metrics, device=device)

confusion = ConfusionMatrix(num_classes=3)
confusion.attach(train_evaluator_swin, 'confusion matrix')
confusion.attach(val_evaluator_swin, 'confusion matrix')

log_interval = 100

def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


trainer_swin.add_event_handler(Events.ITERATION_COMPLETED(every=log_interval), log_training_loss)


def log_training_results(trainer):
    train_evaluator_swin.run(train_dataloader)
    metrics = train_evaluator_swin.state.metrics
    print()
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Training Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Training Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    training_precision_swin.append(precision)
    training_recall_swin.append(recall)
    print()


trainer_swin.add_event_handler(Events.EPOCH_COMPLETED, log_training_results)

def log_validation_results(trainer):
    val_evaluator_swin.run(test_dataloader)
    metrics = val_evaluator_swin.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    precision = metrics['precision'].tolist()
    recall = metrics['recall'].tolist()
    print(f'Validation Precision - Low Nightlight: {precision[0]:.2f} Medium Nightlight: {precision[1]:.2f} High Nightlight: {precision[2]:.2f} ')
    print(f'Validation Recall - Low Nightlight: {recall[0]:.2f} Medium Nightlight: {recall[1]:.2f} High Nightlight: {recall[2]:.2f}')
    print()
    print('Confusion Matrix')
    print(f'{metrics["confusion matrix"]}')
    validation_precision_swin.append(precision)
    validation_recall_swin.append(recall)
    print()

trainer_swin.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)


def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint_swin = ModelCheckpoint(
    "swin_FINAL",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer_swin),
    require_empty=False
)
  
early_stopper = EarlyStopping(patience=6, score_function=score_function, trainer=trainer_swin)

val_evaluator_swin.add_event_handler(Events.COMPLETED, model_checkpoint_swin, {"model": swin_model})
val_evaluator_swin.add_event_handler(Events.COMPLETED, early_stopper)

tb_logger = TensorboardLogger(log_dir="tb-logger-swin_FINAL")

tb_logger.attach_output_handler(
    trainer_swin,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

for tag, evaluator in [("training", train_evaluator_swin), ("validation", val_evaluator_swin)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer_swin)
    )

In [9]:
trainer_swin.run(train_dataloader, max_epochs=7)

Epoch[1], Iter[100] Loss: 0.57
Epoch[1], Iter[200] Loss: 0.51
Epoch[1], Iter[300] Loss: 0.39
Epoch[1], Iter[400] Loss: 0.63
Epoch[1], Iter[500] Loss: 0.73
Epoch[1], Iter[600] Loss: 0.45
Epoch[1], Iter[700] Loss: 0.41
Epoch[1], Iter[800] Loss: 0.78
Epoch[1], Iter[900] Loss: 0.27
Epoch[1], Iter[1000] Loss: 0.42
Epoch[1], Iter[1100] Loss: 0.34
Epoch[1], Iter[1200] Loss: 0.69
Epoch[1], Iter[1300] Loss: 0.87
Epoch[1], Iter[1400] Loss: 0.53
Epoch[1], Iter[1500] Loss: 0.26
Epoch[1], Iter[1600] Loss: 0.32
Epoch[1], Iter[1700] Loss: 0.56
Epoch[1], Iter[1800] Loss: 0.31


c:\Python310\lib\site-packages\ignite\contrib\handlers\base_logger.py:172: UserWarning: Logger output_handler can not log metrics value type <class 'torch.Tensor'>
  warnings.warn(f"Logger output_handler can not log metrics value type {type(value)}")



Training Results - Epoch[1] Avg accuracy: 0.83 Avg loss: 0.40
Training Precision - Low Nightlight: 0.92 Medium Nightlight: 0.54 High Nightlight: 0.71 
Training Recall - Low Nightlight: 0.94 Medium Nightlight: 0.48 High Nightlight: 0.73

Validation Results - Epoch[1] Avg accuracy: 0.85 Avg loss: 0.33
Validation Precision - Low Nightlight: 0.94 Medium Nightlight: 0.48 High Nightlight: 0.73 
Validation Recall - Low Nightlight: 0.94 Medium Nightlight: 0.46 High Nightlight: 0.74

Confusion Matrix
tensor([[271,  14,   2],
        [ 14,  25,  15],
        [  3,  13,  46]])

Epoch[2], Iter[1900] Loss: 0.51
Epoch[2], Iter[2000] Loss: 0.24
Epoch[2], Iter[2100] Loss: 0.51
Epoch[2], Iter[2200] Loss: 0.39
Epoch[2], Iter[2300] Loss: 0.33
Epoch[2], Iter[2400] Loss: 0.91
Epoch[2], Iter[2500] Loss: 0.50
Epoch[2], Iter[2600] Loss: 0.34
Epoch[2], Iter[2700] Loss: 0.41
Epoch[2], Iter[2800] Loss: 0.46
Epoch[2], Iter[2900] Loss: 0.61
Epoch[2], Iter[3000] Loss: 0.54
Epoch[2], Iter[3100] Loss: 0.10
Epoch[2],

State:
	iteration: 12635
	epoch: 7
	epoch_length: 1805
	max_epochs: 7
	output: 0.36851072311401367
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>